# Energy Allocation

In [52]:
#using Pkg
#Pkg.add("XLSX")

In [1]:
using XLSX, Gurobi, StatsBase, CSV, DataFrames, 
JuMP, LinearAlgebra, Distributions, Random,
GLMNet, ScikitLearn, MLBase, CategoricalArrays, Plots,
Dates, Clustering, Distances, StatsPlots, ProgressMeter, 
StableRNGs, ParallelKMeans, JuMP, Gurobi

In [2]:
# Read xlsm file
initial_df = DataFrame(XLSX.readtable("eCO2mix_RTE_Annuel-Definitif_2020.xlsm", "eCO2mix_RTE_Annuel-Definitif_20"));

In [3]:
# Let's clean the data

# Remove all rows with missing values
df = dropmissing(initial_df, disallowmissing=true);

# remove every columns before the Date column and after the Ech. comm. Allemagne-Belgique column
df = df[:, 3:23]

# put the columns PrÈvision J-1, PrÈvision J, Pompage, Ech. physiques
prev_pomp_df = df[:, [4, 5, 13, 15]]
# drom column 4,5,13,15
df = df[:, Not([4, 5, 13, 15])]

# rename the columns
rename!(df, Symbol.(["Date", "Time", "Consumption", "Fuel", "Coal", "Gas", "Nuclear", "Wind", "Solar", "Hydraulic", "Bioenergy", "CO2", "England", "Spain", "Italy", "Switzerland", "Germany_Belgium"]))

# for each country, we will divide in import and export, depending on the sign of the corresponding column

neighbors_columns = ["England", "Spain", "Italy", "Switzerland", "Germany_Belgium"]
for col in neighbors_columns
    # create a new import column, which values are positive if the original column is positive, and 0 otherwise
    df[!, Symbol(col * "_import")] = max.(df[!, Symbol(col)], 0)
    # create a new export column, which values are positive if the original column is negative, and 0 otherwise
    df[!, Symbol(col * "_export")] = max.(-df[!, Symbol(col)], 0)
end

# drop the original columns
df = df[:, Not(neighbors_columns)]


# convert the date column to a string
df[!, :Date] = string.(df[!, :Date])
# convert the time column to a string
df[!, :Time] = string.(df[!, :Time])
# merge the date and time columns into a single column
df[!, :DateTime] = df[!, :Date] .* " " .* df[!, :Time]
# convert the DateTime column to a DateTime type
df[!, :DateTime] = DateTime.(df[!, :DateTime], "yyyy-mm-dd HH:MM:SS")
# convert date and time columns to a DateTime type
df[!, :Date] = Date.(df[!, :Date], "yyyy-mm-dd")
df[!, :Time] = Time.(df[!, :Time], "HH:MM:SS")

# sort the dataframe by DateTime
df = sort(df, :DateTime)
# put the DateTime column as the first column
df = df[:, [end; 1:end-1]]

# the units are in MWh
# convert the Any type to Float64
df[!, :Consumption] = convert(Array{Float64}, df[!, :Consumption])
df[!, :Fuel] = convert(Array{Float64}, df[!, :Fuel])
df[!, :Coal] = convert(Array{Float64}, df[!, :Coal])
df[!, :Gas] = convert(Array{Float64}, df[!, :Gas])
df[!, :Nuclear] = convert(Array{Float64}, df[!, :Nuclear])
df[!, :Wind] = convert(Array{Float64}, df[!, :Wind])
df[!, :Solar] = convert(Array{Float64}, df[!, :Solar])
df[!, :Hydraulic] = convert(Array{Float64}, df[!, :Hydraulic])
df[!, :Bioenergy] = convert(Array{Float64}, df[!, :Bioenergy])
df[!, :CO2] = convert(Array{Float64}, df[!, :CO2])
df[!, :England_import] = convert(Array{Float64}, df[!, :England_import])
df[!, :England_export] = convert(Array{Float64}, df[!, :England_export])
df[!, :Spain_import] = convert(Array{Float64}, df[!, :Spain_import])
df[!, :Spain_export] = convert(Array{Float64}, df[!, :Spain_export])
df[!, :Italy_import] = convert(Array{Float64}, df[!, :Italy_import])
df[!, :Italy_export] = convert(Array{Float64}, df[!, :Italy_export])
df[!, :Switzerland_import] = convert(Array{Float64}, df[!, :Switzerland_import])
df[!, :Switzerland_export] = convert(Array{Float64}, df[!, :Switzerland_export])
df[!, :Germany_Belgium_import] = convert(Array{Float64}, df[!, :Germany_Belgium_import])
df[!, :Germany_Belgium_export] = convert(Array{Float64}, df[!, :Germany_Belgium_export]);


### Pre-processing

In [4]:
function transform_data(df, push_nuclear=0.2, push_solar=0.15)
    inc_nuc = 1 + push_nuclear
    inc_sol = 1 + push_solar
    
    # get max value of nuclear production overall
    max_nuclear = maximum(df[!, :Nuclear])
    df[!, :Nuclear_cap] = fill(max_nuclear* inc_nuc, size(df[!, :Nuclear]))
    max_hydraulic = maximum(df[!, :Hydraulic])
    df[!, :Hydraulic_cap] = fill(max_hydraulic* inc_nuc, size(df[!, :Hydraulic]))
    max_bioenergy = maximum(df[!, :Bioenergy])
    df[!, :Bioenergy_cap] = fill(max_bioenergy* inc_nuc, size(df[!, :Bioenergy]))


    df[!, :Week] = Dates.week.(df[!, :Date])
    # get the average production of gas, coal, fuel, solar and wind for each week of the year in one dataframe
    df_weekly = combine(groupby(df, :Week), :Solar => mean => :Solar_cap, :Wind => mean => :Wind_cap, :Gas => mean => :Gas_cap, :Coal => mean => :Coal_cap, :Fuel => mean => :Fuel_cap)
    # sort the dataframe by week
    df_weekly = sort(df_weekly, :Week)
    # add 20% to each column of df_weekly
    df_weekly[!, :Solar_cap] = df_weekly[!, :Solar_cap] * inc_sol
    df_weekly[!, :Wind_cap] = df_weekly[!, :Wind_cap] * inc_sol
    df_weekly[!, :Gas_cap] = df_weekly[!, :Gas_cap] * inc_sol
    df_weekly[!, :Coal_cap] = df_weekly[!, :Coal_cap] * inc_sol
    df_weekly[!, :Fuel_cap] = df_weekly[!, :Fuel_cap] * inc_sol
    # sort the dataframe by week
    df_weekly = sort(df_weekly, :Week)
    # merge the two dataframes df and df_weekly on week column
    df = leftjoin(df, df_weekly, on=:Week)

    cost_nuclear = 59.8:2:109.8
    cost_wind = 90:1:90
    cost_hydraulic = 15:0.5:20
    cost_solar = 142:1:142
    cost_gas = 70:2:100
    cost_coal = 100:1:100
    cost_fuel = 86:1:86
    cost_bioenergy = 85:2:122

    # create a vector of size 53 for each type of energy cost with random values in the range defined above
    cost_nuclear = rand(cost_nuclear, 53)
    # vary the costs with a random number in the std of the range of values
    cost_nuclear = cost_nuclear .+ randn(53) * std(cost_nuclear)

    cost_hydraulic = rand(cost_hydraulic, 53)
    cost_hydraulic = cost_hydraulic .+ randn(53) * std(cost_hydraulic)

    cost_gas = rand(cost_gas, 53)
    cost_gas = cost_gas .+ randn(53) * std(cost_gas)

    cost_bioenergy = rand(cost_bioenergy, 53)
    cost_bioenergy = cost_bioenergy .+ randn(53) * std(cost_bioenergy)

    cost_wind = rand(cost_wind, 53)
    cost_wind = cost_wind .+ randn(53) * std(cost_wind)

    cost_solar = rand(cost_solar, 53)
    cost_solar = cost_solar .+ randn(53) * std(cost_solar)

    cost_coal = rand(cost_coal, 53)
    cost_coal = cost_coal .+ randn(53) * std(cost_coal)

    cost_fuel = rand(cost_fuel, 53)
    cost_fuel = cost_fuel .+ randn(53) * std(cost_fuel)

    # create a dataframe with the cost of each type of energy
    df_cost = DataFrame(Week = 1:53, Nuclear_cost = cost_nuclear, Wind_cost = cost_wind, Solar_cost = cost_solar, Gas_cost = cost_gas, Coal_cost = cost_coal, Fuel_cost = cost_fuel, Hydraulic_cost = cost_hydraulic, Bioenergy_cost = cost_bioenergy)

    # sort the dataframe by week
    df_cost = sort(df_cost, :Week)

    # merge the two dataframes df and df_cost on week column
    df = leftjoin(df, df_cost, on=:Week)

    

    df[!, :Total_cost] = (df[!, :Nuclear] .* df[!, :Nuclear_cost] .+ df[!, :Wind] .* df[!, :Wind_cost] .+ df[!, :Solar] .* df[!, :Solar_cost] .+ df[!, :Gas] .* df[!, :Gas_cost] .+ df[!, :Coal] .* df[!, :Coal_cost] .+ df[!, :Fuel] .* df[!, :Fuel_cost] .+ df[!, :Hydraulic] .* df[!, :Hydraulic_cost] .+ df[!, :Bioenergy] .* df[!, :Bioenergy_cost]) ./ (df[!, :Nuclear] .+ df[!, :Wind] .+ df[!, :Solar] .+ df[!, :Gas] .+ df[!, :Coal] .+ df[!, :Fuel] .+ df[!, :Hydraulic] .+ df[!, :Bioenergy])
    # Create a column called "Export_Price" which is the Total_cost + (10 plus or minus 3%, randomly)
    df[!, :Export_Price] = df[!, :Total_cost] .+ (df[!, :Total_cost] .* (0.1 .+ randn(size(df)[1]) * 0.03))

    # Create a column called "England_Price"
    # It is equal to 1000 if England_import==0 and Export_Price minus (10% plus or minus 3%) otherwise
    df[!, :England_Price] = ifelse.(df[!, :England_import] .== 0, 1000, df[!, :Export_Price] .- (df[!, :Export_Price] .* (0.1 .+ randn(size(df)[1]) * 0.03)))
    df[!, :Spain_Price] = ifelse.(df[!, :Spain_import] .== 0, 1000, df[!, :Export_Price] .- (df[!, :Export_Price] .* (0.15 .+ randn(size(df)[1]) * 0.03)))
    df[!, :Italy_Price] = ifelse.(df[!, :Italy_import] .== 0, 1000, df[!, :Export_Price] .- (df[!, :Export_Price] .* (0.07 .+ randn(size(df)[1]) * 0.03)))
    df[!, :Switzerland_Price] = ifelse.(df[!, :Switzerland_import] .== 0, 1000, df[!, :Export_Price] .- (df[!, :Export_Price] .* (0.1 .+ randn(size(df)[1]) * 0.03)))
    df[!, :Germany_Belgium_Price] = ifelse.(df[!, :Germany_Belgium_import] .== 0, 1000, df[!, :Export_Price] .- (df[!, :Export_Price] .* (0.2 .+ randn(size(df)[1]) * 0.03)))

    return df
end

transform_data (generic function with 3 methods)

### Eco-cost synthetic data

For each source s, at each time t, we need an Eco-cost $f_{s,t}$

In [5]:
Eco_cost_footprint = [0.0347, 0.0003, 0.0175, 0.0003, 0.0281, 0.0002, 0.0030, 0.0164, 0.0130, 0.0193, 0.0172, 0.0067, 0.0226, 0.0065, 0.0109, 0.0024, 0.0136, 0.0099, 0.0154, 0.0270, 0.0099, 0.0095, 0.0084, 0.0006, 0.0091, 0.0156]
carbon_footprint_equiv = [0.30, 0.0023, 0.15, 0.00, 0.24, 0.0017, 0.03, 0.14, 0.11, 0.17, 0.15, 0.06, 0.19, 0.06, 0.09, 0.02, 0.12, 0.09, 0.13, 0.23, 0.08, 0.08, 0.07, 0.01, 0.08, 0.13]

# compute the mean of carbon_footprint_equiv / Eco_cost_footprint
mean_Eco_cost_carbon_footprint_equiv =  1 / mean(carbon_footprint_equiv ./ Eco_cost_footprint)

0.11652081241930631

In [6]:
function compute_eco_costs(df)

    # compute mean of CO2 column in different units
    CO2 = df[!, :CO2]
    mean_CO2 = mean(CO2)
    mean_CO2_GJ = mean_CO2 * 3.6

    # add values human_health, eco_toxicity, resource_scarcity, carbon_footprint for each source of energy, based on US Idemat Data
    fuel_info = [0.00246, 0.00821, 0.00007, 0.0281]
    coal_info = [0.00340, 0.00497, 0.00046, 0.0347]
    gas_info = [0.00324, 0.00294, 0.00013, 0.0175]
    nuclear_info = [0.00008, 0.00051, 0.03812, 0.0003]

    # we approximate all renewable energy sources with the same values
    hydraulic_info = [0.00002, 0.00003, 0.00017, 0.0002]
    bioenergy_info = [0.00002, 0.00003, 0.00017, 0.0002]
    wind_info = [0.00002, 0.00003, 0.00017, 0.0002]
    solar_info = [0.00002, 0.00003, 0.00017, 0.0002]

    # get the mean production of each source of energy
    mean_hydraulic = mean(df[!, :Hydraulic])
    mean_bioenergy = mean(df[!, :Bioenergy])
    mean_wind = mean(df[!, :Wind])
    mean_solar = mean(df[!, :Solar])
    mean_nuclear = mean(df[!, :Nuclear])
    mean_gas = mean(df[!, :Gas])
    mean_coal = mean(df[!, :Coal])
    mean_fuel = mean(df[!, :Fuel])

    # get the percentage of use of each source of energy
    total_production = mean_hydraulic + mean_bioenergy + mean_wind + mean_solar + mean_nuclear + mean_gas + mean_coal + mean_fuel
    percentage_hydraulic = mean_hydraulic / total_production
    percentage_bioenergy = mean_bioenergy / total_production
    percentage_wind = mean_wind / total_production
    percentage_solar = mean_solar / total_production
    percentage_nuclear = mean_nuclear / total_production
    percentage_gas = mean_gas / total_production
    percentage_coal = mean_coal / total_production
    percentage_fuel = mean_fuel / total_production

    # sum infos using the percentage of use of each source of energy
    sum_info = percentage_hydraulic * hydraulic_info + percentage_bioenergy * bioenergy_info + percentage_wind * wind_info + percentage_solar * solar_info + percentage_nuclear * nuclear_info + percentage_gas * gas_info + percentage_coal * coal_info + percentage_fuel * fuel_info;


    idemat_eco_cost_per_GJ = 18.19 # in euros/GJ
    idemat_carbon_footprint_per_GJ = 20.40 # in kg/GJ

    idemat_eco_costs = [0.21, 0.78, 14.83, 2.37] # in euros/GJ
    idemat_proportions = idemat_eco_costs / sum(idemat_eco_costs)
    # for each value in sum_info, determine the percentage of use of each source of energy

    # percentages of impact of each source of energy on eco_cost
    percentages_hydraulic = percentage_hydraulic .* hydraulic_info ./ sum_info .* idemat_proportions
    percentages_bioenergy = percentage_bioenergy .* bioenergy_info ./ sum_info .* idemat_proportions
    percentages_wind = percentage_wind .* wind_info ./ sum_info .* idemat_proportions
    percentages_solar = percentage_solar .* solar_info ./ sum_info .* idemat_proportions
    percentages_nuclear = percentage_nuclear .* nuclear_info ./ sum_info .* idemat_proportions
    percentages_gas = percentage_gas .* gas_info ./ sum_info .* idemat_proportions
    percentages_coal = percentage_coal .* coal_info ./ sum_info .* idemat_proportions
    percentages_fuel = percentage_fuel .* fuel_info ./ sum_info .* idemat_proportions


    idemat_eco_cost_per_MWh = idemat_eco_cost_per_GJ  / 3.6
    idemat_carbon_footprint_per_MWh = idemat_carbon_footprint_per_GJ / 3.6

    real_eco_cost_per_MWh = idemat_eco_cost_per_MWh / idemat_carbon_footprint_per_MWh * mean_CO2
    real_eco_costs = idemat_proportions * real_eco_cost_per_MWh # in euros/MWh

    # in euros/MWh
    eco_costs_hydraulic = percentages_hydraulic .* real_eco_cost_per_MWh
    eco_costs_bioenergy = percentages_bioenergy .* real_eco_cost_per_MWh
    eco_costs_wind = percentages_wind .* real_eco_cost_per_MWh
    eco_costs_solar = percentages_solar .* real_eco_cost_per_MWh
    eco_costs_nuclear = percentages_nuclear .* real_eco_cost_per_MWh
    eco_costs_gas = percentages_gas .* real_eco_cost_per_MWh
    eco_costs_coal = percentages_coal .* real_eco_cost_per_MWh
    eco_costs_fuel = percentages_fuel .* real_eco_cost_per_MWh

    Hydraulic_Eco_cost = sum(eco_costs_hydraulic)
    Bioenergy_Eco_cost = sum(eco_costs_bioenergy)
    Wind_Eco_cost = sum(eco_costs_wind)
    Solar_Eco_cost = sum(eco_costs_solar)
    Nuclear_Eco_cost = sum(eco_costs_nuclear)
    Gas_Eco_cost = sum(eco_costs_gas)
    Coal_Eco_cost = sum(eco_costs_coal)
    Fuel_Eco_cost = sum(eco_costs_fuel)

    # create a little dataframe with the results
    eco_costs = DataFrame(Source = ["Hydraulic", "Bioenergy", "Wind", "Solar", "Nuclear", "Gas", "Coal", "Fuel"],
        Eco_cost = [Hydraulic_Eco_cost, Bioenergy_Eco_cost, Wind_Eco_cost, Solar_Eco_cost, Nuclear_Eco_cost, Gas_Eco_cost, Coal_Eco_cost, Fuel_Eco_cost])
    
    # sort the dataframe by the eco_cost
    eco_costs = eco_costs[sortperm(eco_costs.Eco_cost), :]

    # return the dataframe
    return eco_costs
end

compute_eco_costs (generic function with 1 method)

In [7]:
function compute_carbon_footprints(df)
    # in kg/MWh
    nuclear_footprint_estimates = [0.006, 0.006] * 1000
    hydraulic_footprint_estimates = [0.006, 0.004] * 1000
    wind_footprint_estimates = [0.0141, 0.0073] * 1000
    solar_footprint_estimates = [0.0439, 0.055] * 1000
    bioenergy_footprint_estimates = [0.045, 0.045] * 1000
    gas_footprint_estimates = [0.418, 0.406] * 1000
    fuel_footprint_estimates = [0.73, 0.704] * 1000
    coal_footprint_estimates = [1.06, 1.038] * 1000

    # store the mean of these estimates in a dataframe
    mean_nuclear_footprint = mean(nuclear_footprint_estimates)
    mean_hydraulic_footprint = mean(hydraulic_footprint_estimates)
    mean_wind_footprint = mean(wind_footprint_estimates)
    mean_solar_footprint = mean(solar_footprint_estimates)
    mean_bioenergy_footprint = mean(bioenergy_footprint_estimates)
    mean_gas_footprint = mean(gas_footprint_estimates)
    mean_fuel_footprint = mean(fuel_footprint_estimates)
    mean_coal_footprint = mean(coal_footprint_estimates)

    # store the mean of these estimates in a dataframe
    mean_footprints = DataFrame(Source = ["Hydraulic", "Bioenergy", "Wind", "Solar", "Nuclear", "Gas", "Coal", "Fuel"],
        Mean_footprint = [mean_hydraulic_footprint, mean_bioenergy_footprint, mean_wind_footprint, mean_solar_footprint, mean_nuclear_footprint, mean_gas_footprint, mean_coal_footprint, mean_fuel_footprint])
    
    # sort the dataframe by the mean_footprint
    mean_footprints = mean_footprints[sortperm(mean_footprints.Mean_footprint), :]

    # return the dataframe
    return mean_footprints
end


compute_carbon_footprints (generic function with 1 method)

In [8]:
function compute_neighbors_footprints(df)

    # compute carbon footprint for other countries
    idemat_France_carbon_footprint = 20.40 # in Kg/GJ
    idemat_England_carbon_footprint = 77.80 # in Kg/GJ
    idemat_Spain_carbon_footprint = 72.00 # in Kg/GJ
    idemat_Italy_carbon_footprint = 83.81 # in Kg/GJ
    idemat_Switzerland_carbon_footprint = 5.57 # in Kg/GJ
    idemat_Germany_carbon_footprint = 116.92 # in Kg/GJ
    idemat_Belgium_carbon_footprint = 57.48 # in Kg/GJ

    idemat_France_eco_cost = 18.19 # in euros/GJ
    idemat_England_eco_cost = 14.16 # in euros/GJ
    idemat_Spain_eco_cost = 15.54 # in euros/GJ
    idemat_Italy_eco_cost = 10.78 # in euros/GJ
    idemat_Switzerland_eco_cost = 8.85 # in euros/GJ
    idemat_Germany_eco_cost = 18.48 # in euros/GJ
    idemat_Belgium_eco_cost = 17.14 # in euros/GJ

    idemat_Germany_Belgium_carbon_footprint = 580 / 673 * idemat_Germany_carbon_footprint  + 93 / 673 * idemat_Belgium_carbon_footprint

    England_proportional_footprint = idemat_England_carbon_footprint / idemat_France_carbon_footprint
    Spain_proportional_footprint = idemat_Spain_carbon_footprint / idemat_France_carbon_footprint
    Italy_proportional_footprint = idemat_Italy_carbon_footprint / idemat_France_carbon_footprint
    Switzerland_proportional_footprint = idemat_Switzerland_carbon_footprint / idemat_France_carbon_footprint
    Germany_Belgium_proportional_footprint = idemat_Germany_Belgium_carbon_footprint / idemat_France_carbon_footprint

    mean_Hydraulic = mean(df[!, :Hydraulic])
    mean_Bioenergy = mean(df[!, :Bioenergy])
    mean_Wind = mean(df[!, :Wind])
    mean_Solar = mean(df[!, :Solar])
    mean_Nuclear = mean(df[!, :Nuclear])
    mean_Gas = mean(df[!, :Gas])
    mean_Coal = mean(df[!, :Coal])
    mean_Fuel = mean(df[!, :Fuel])

    mean_total = mean_Hydraulic + mean_Bioenergy + mean_Wind + mean_Solar + mean_Nuclear + mean_Gas + mean_Coal + mean_Fuel

    carbon_footprints_df = compute_carbon_footprints(df) # in Kg/MWh
    total_carbon_footprint = 0
    for source in carbon_footprints_df.Source
        percentage = mean(df[!, Symbol(source)]) / mean_total
        total_carbon_footprint += carbon_footprints_df[carbon_footprints_df.Source .== source, :Mean_footprint][1] * percentage
    end

    England_carbon_footprint = total_carbon_footprint * England_proportional_footprint
    Spain_carbon_footprint = total_carbon_footprint * Spain_proportional_footprint
    Italy_carbon_footprint = total_carbon_footprint * Italy_proportional_footprint
    Switzerland_carbon_footprint = total_carbon_footprint * Switzerland_proportional_footprint
    Germany_Belgium_carbon_footprint = total_carbon_footprint * Germany_Belgium_proportional_footprint

    # return a dataframe with the carbon footprint of the neighbors
    neighbors_carbon_footprints = DataFrame(Country = ["England", "Spain", "Italy", "Switzerland", "Germany_Belgium"],
        Carbon_footprint = [England_carbon_footprint, Spain_carbon_footprint, Italy_carbon_footprint, Switzerland_carbon_footprint, Germany_Belgium_carbon_footprint])

    # sort the dataframe by the carbon_footprint
    neighbors_carbon_footprints = neighbors_carbon_footprints[sortperm(neighbors_carbon_footprints.Carbon_footprint), :]

    # return the dataframe
    return neighbors_carbon_footprints

end

compute_neighbors_footprints (generic function with 1 method)

## Optimization Formulation

### Baseline Scenario

In [9]:
df_baseline = transform_data(df)

Row,DateTime,Date,Time,Consumption,Fuel,Coal,Gas,Nuclear,Wind,Solar,Hydraulic,Bioenergy,CO2,England_import,England_export,Spain_import,Spain_export,Italy_import,Italy_export,Switzerland_import,Switzerland_export,Germany_Belgium_import,Germany_Belgium_export,Nuclear_cap,Hydraulic_cap,Bioenergy_cap,Week,Solar_cap,Wind_cap,Gas_cap,Coal_cap,Fuel_cap,Nuclear_cost,Wind_cost,Solar_cost,Gas_cost,Coal_cost,Fuel_cost,Hydraulic_cost,Bioenergy_cost,Total_cost,Export_Price,England_Price,Spain_Price,Italy_Price,Switzerland_Price,Germany_Belgium_Price
,DateTime,Date,Time,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Real,Real,Real,Real,Real
1,2020-01-01T00:00:00,2020-01-01,00:00:00,67068.0,300.0,14.0,6515.0,49226.0,3748.0,0.0,9084.0,1185.0,40.0,0.0,1704.0,0.0,226.0,0.0,1954.0,0.0,2267.0,1763.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,98.3443,108.444,1000,1000,1000,1000,88.5942
2,2020-01-01T00:30:00,2020-01-01,00:30:00,66103.0,107.0,13.0,6692.0,49436.0,3627.0,0.0,9586.0,1194.0,39.0,0.0,1704.0,0.0,226.0,0.0,1954.0,0.0,2267.0,1763.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,97.8694,106.127,1000,1000,1000,1000,84.588
3,2020-01-01T01:00:00,2020-01-01,01:00:00,63943.0,106.0,13.0,6257.0,48970.0,3203.0,0.0,9133.0,1188.0,37.0,0.0,1704.0,358.0,0.0,0.0,1867.0,0.0,2527.0,1035.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,98.3552,107.569,1000,95.4712,1000,1000,85.5191
4,2020-01-01T01:30:00,2020-01-01,01:30:00,63904.0,107.0,14.0,5630.0,49648.0,2981.0,0.0,9124.0,1187.0,34.0,0.0,1704.0,358.0,0.0,0.0,1867.0,0.0,2527.0,1085.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,98.6179,110.95,1000,97.423,1000,1000,92.2449
5,2020-01-01T02:00:00,2020-01-01,02:00:00,63408.0,107.0,14.0,5337.0,49764.0,2799.0,0.0,8699.0,1198.0,33.0,0.0,1704.0,301.0,0.0,0.0,1480.0,0.0,2463.0,1668.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,99.1857,111.537,1000,96.2025,1000,1000,89.5986
6,2020-01-01T02:30:00,2020-01-01,02:30:00,62711.0,107.0,15.0,4495.0,49747.0,2763.0,0.0,8212.0,1184.0,29.0,0.0,1704.0,301.0,0.0,0.0,1480.0,0.0,2463.0,1761.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,99.8648,107.581,1000,91.7211,1000,1000,83.1744
7,2020-01-01T03:00:00,2020-01-01,03:00:00,60825.0,107.0,14.0,3660.0,49527.0,2810.0,0.0,7658.0,1185.0,26.0,0.0,1704.0,614.0,0.0,0.0,1400.0,0.0,2350.0,595.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,100.593,109.624,1000,92.5093,1000,1000,84.7796
8,2020-01-01T03:30:00,2020-01-01,03:30:00,59332.0,107.0,14.0,2942.0,49887.0,2822.0,0.0,7335.0,1194.0,22.0,0.0,1704.0,614.0,0.0,0.0,1400.0,0.0,2350.0,499.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,101.174,111.07,1000,92.3101,1000,1000,86.4761
9,2020-01-01T04:00:00,2020-01-01,04:00:00,58004.0,107.0,13.0,2942.0,49299.0,2752.0,0.0,7021.0,1201.0,23.0,0.0,1704.0,1109.0,0.0,0.0,969.0,0.0,2402.0,237.0,0.0,65664.0,21172.8,1656.0,1,630.703,3975.33,7326.91,16.6223,125.067,114.636,90.0,142.0,91.2606,100.0,86.0,20.6674,85.4757,101.458,106.865,1000,93.172,1000,1000,86.9004


In [10]:
# for each source, add coefficient lambda = p * cost to produce 1 MWh of energy with this source
lambda_factor = 10
df_baseline[!, :Lambda_Hydraulic] = lambda_factor .* df_baseline[!, :Hydraulic_cost]
df_baseline[!, :Lambda_Bioenergy] = lambda_factor .* df_baseline[!, :Bioenergy_cost]
df_baseline[!, :Lambda_Wind] = lambda_factor .* df_baseline[!, :Wind_cost]
df_baseline[!, :Lambda_Solar] = lambda_factor .* df_baseline[!, :Solar_cost]
df_baseline[!, :Lambda_Nuclear] = lambda_factor .* df_baseline[!, :Nuclear_cost]
df_baseline[!, :Lambda_Gas] = lambda_factor .* df_baseline[!, :Gas_cost]
df_baseline[!, :Lambda_Coal] = lambda_factor .* df_baseline[!, :Coal_cost]
df_baseline[!, :Lambda_Fuel] = lambda_factor .* df_baseline[!, :Fuel_cost];

In [45]:
# some information
sources = ["Hydraulic", "Bioenergy", "Wind", "Solar", "Nuclear", "Gas", "Coal", "Fuel"]
neighbors = ["England", "Spain", "Italy", "Switzerland", "Germany_Belgium"]

function lambda_baseline_cost(x, y, z, df_base)
    # for each row of the dataframe, compute a cost
    # x is the amount of energy produced by each source
    # y is the amount of energy imported from each country
    # z is the amount of energy exported to each country
    T = size(df_base, 1)

    # add production cost
    production_cost = 0
    for i in 1:size(sources, 1)
        # add cost of producing energy with each source
        production_cost += sum(x[:, i] .* df_base[!, Symbol(sources[i] * "_cost")])
        # add lambda cost of exceeding maximum cap
        production_cost += sum(x[:, i] .- df_base[!, Symbol(sources[i] * "_cap")] .* df_base[!, Symbol("Lambda_" * sources[i])])
    end

    # add import cost
    import_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of importing energy from each neighbor
        import_cost += sum(y[:, i] .* df_base[!, Symbol(neighbors[i] * "_Price")])
    end

    # add export cost
    export_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of exporting energy to each neighbor
        export_cost += sum(z[:, i] .* df_base[!, "Export_Price"])
    end    

    # return the total cost
    return production_cost + import_cost + export_cost
end


function real_baseline_cost(x, y, z, df_base)
    # for each row of the dataframe, compute a cost
    # x is the amount of energy produced by each source
    # y is the amount of energy imported from each country
    # z is the amount of energy exported to each country

    # add production cost
    production_cost = 0
    for i in 1:size(sources, 1)
        # add cost of producing energy with each source
        production_cost += sum(x[:, i] .* df_base[!, Symbol(sources[i] * "_cost")])
    end

    # add import cost
    import_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of importing energy from each neighbor
        import_cost += sum(y[:, i] .* df_base[!, Symbol(neighbors[i] * "_Price")])
    end

    # add export revenue
    export_revenue = 0
    for i in 1:size(neighbors, 1)
        # add cost of exporting energy to each neighbor
        export_revenue += sum(z[:, i] .* df_base[!, "Export_Price"])
    end    

    # return the total cost
    return production_cost + import_cost - export_revenue
end


#function to convert the optimal values to a dataframe
function convert_to_df(x, y, z, df_base)
    # create a new dataframe
    df = DataFrame()

    # add the time column
    df[!, :Time] = df_base[!, :Time]

    # add the production columns
    for i in 1:size(sources, 1)
        df[!, Symbol(sources[i])] = x[:, i]
    end

    # add the import columns
    for i in 1:size(neighbors, 1)
        df[!, Symbol(neighbors[i] * "_import")] = y[:, i]
    end

    # add the export columns
    for i in 1:size(neighbors, 1)
        df[!, Symbol(neighbors[i] * "_export")] = z[:, i]
    end

    # return the dataframe
    return df
end


convert_to_df (generic function with 1 method)

In [38]:
# function to build a Gurobi Optimizer to optimize the baseline_cost
function baseline_optimizer(df_base)

    # create a Gurobi Optimizer
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    # add variables
    T = size(df_base, 1)
    @variable(model, x[1:T, 1:8] >= 0) # production of energy with each source
    @variable(model, y[1:T, 1:5] >= 0) # import of energy from each neighbor
    @variable(model, z[1:T, 1:5] >= 0) # export of energy to each neighbor

    # add objective
    #@objective(model, Min, lambda_baseline_cost(x, y, z, df_base))
    @objective(model, Min, real_baseline_cost(x, y, z, df_base))


    # add constraints
    # Demand constraint, we need to produce enough energy for France
    @constraint(model, [t in 1:T], sum(x[t, :]) + sum(y[t, :]) - sum(z[t, :]) >= df_base[t, :Consumption])

    # maximum import available constraint
    @constraint(model, [t in 1:T, i in 1:5], y[t, i] <= df_base[t, Symbol(neighbors[i] * "_import")])

    # maximum export available constraint
    @constraint(model, [t in 1:T, i in 1:5], z[t, i] <= df_base[t, Symbol(neighbors[i] * "_export")])

    # maximum production constraint
    @constraint(model, [t in 1:T, i in 1:8], x[t, i] <= df_base[t, Symbol(sources[i] * "_cap")])

    
    # optimize
    optimize!(model)

    # return the optimizer
    return model
end

baseline_optimizer (generic function with 1 method)

In [39]:
model_1 = baseline_optimizer(df_baseline);

# get the optimal values
x_opt = value.(model_1[:x])
y_opt = value.(model_1[:y])
z_opt = value.(model_1[:z])

# convert the optimal values to a dataframe
df_opt = convert_to_df(x_opt, y_opt, z_opt, df_baseline)


# compute the cost
println("Lambda Baseline cost: ", lambda_baseline_cost(x_opt, y_opt, z_opt, df_baseline))
println("Real baseline cost: ", real_baseline_cost(x_opt, y_opt, z_opt, df_baseline));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Lambda Baseline cost: -1.1973843970551196e12
Real baseline cost: 4.811987163800417e10


In [40]:
df_opt

Row,Time,Hydraulic,Bioenergy,Wind,Solar,Nuclear,Gas,Coal,Fuel,England_import,Spain_import,Italy_import,Switzerland_import,Germany_Belgium_import,England_export,Spain_export,Italy_export,Switzerland_export,Germany_Belgium_export
,Time,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,00:00:00,21172.8,1656.0,3975.33,0.0,32795.3,7326.91,16.6223,125.067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00:30:00,21172.8,1656.0,3975.33,0.0,31830.3,7326.91,16.6223,125.067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01:00:00,21172.8,1656.0,3975.33,0.0,29312.3,7326.91,16.6223,125.067,0.0,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01:30:00,21172.8,1656.0,3975.33,0.0,29273.3,7326.91,16.6223,125.067,0.0,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,02:00:00,21172.8,1656.0,3975.33,0.0,28834.3,7326.91,16.6223,125.067,0.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,02:30:00,21172.8,1656.0,3975.33,0.0,28137.3,7326.91,16.6223,125.067,0.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,03:00:00,21172.8,1656.0,3975.33,0.0,25938.3,7326.91,16.6223,125.067,0.0,614.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,03:30:00,21172.8,1656.0,3975.33,0.0,24445.3,7326.91,16.6223,125.067,0.0,614.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,04:00:00,21172.8,1656.0,3975.33,0.0,22622.3,7326.91,16.6223,125.067,0.0,1109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Crisis scenario

In [46]:
function crisis_df(df, boost_coef)
    # make all import columns of Switzerland Italy Germany_Belgium and England to zero
    df[!, :Switzerland_import] .= 0
    df[!, :Italy_import] .= 0
    df[!, :Germany_Belgium_import] .= 0
    df[!, :England_import] .= 0

    # boost export demand by 50%
    df[!, :Switzerland_export] .= df[!, :Switzerland_export] * boost_coef
    df[!, :Italy_export] .= df[!, :Italy_export] * boost_coef
    df[!, :Germany_Belgium_export] .= df[!, :Germany_Belgium_export] * boost_coef
    df[!, :England_export] .= df[!, :England_export] * boost_coef
    
    return df
end

crisis_df (generic function with 1 method)

In [47]:
df_crisis = crisis_df(df_baseline, 2);

In [53]:
# function very similar to baseline_optimizer, but with an additional constraint
function crisis_optimizer(df_base, alpha)

    # create a Gurobi Optimizer
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    # add variables
    T = size(df_base, 1)
    @variable(model, x[1:T, 1:8] >= 0) # production of energy with each source
    @variable(model, y[1:T, 1:5] >= 0) # import of energy from each neighbor
    @variable(model, z[1:T, 1:5] >= 0) # export of energy to each neighbor

    # add objective
    #@objective(model, Min, lambda_baseline_cost(x, y, z, df_base))
    @objective(model, Min, real_baseline_cost(x, y, z, df_base))


    # add constraints
    # Demand constraint, we need to produce enough energy for France
    @constraint(model, [t in 1:T], sum(x[t, :]) + sum(y[t, :]) - sum(z[t, :]) >= df_base[t, :Consumption])

    # maximum import available constraint
    @constraint(model, [t in 1:T, i in 1:5], y[t, i] <= df_base[t, Symbol(neighbors[i] * "_import")])

    # maximum export available constraint
    @constraint(model, [t in 1:T, i in 1:5], z[t, i] <= df_base[t, Symbol(neighbors[i] * "_export")])

    # maximum production constraint
    @constraint(model, [t in 1:T, i in 1:8], x[t, i] <= df_base[t, Symbol(sources[i] * "_cap")])

    # crisis constraint, we need to satisfy at least alpha percent of the neighbors demand
    @constraint(model, [t in 1:T, i in 1:5], z[t, i] >= alpha * df_base[t, Symbol(neighbors[i] * "_export")])

    
    # optimize
    optimize!(model)

    # return the optimizer
    return model
end

crisis_optimizer (generic function with 1 method)

In [56]:
model_2 = crisis_optimizer(df_crisis, 0.1);

# get the optimal values
x_opt = value.(model_2[:x])
y_opt = value.(model_2[:y])
z_opt = value.(model_2[:z])

# convert the optimal values to a dataframe
df_crisis_opt = convert_to_df(x_opt, y_opt, z_opt, df_crisis)


# compute the cost
println("Lambda Baseline cost: ", lambda_baseline_cost(x_opt, y_opt, z_opt, df_crisis))
println("Real baseline cost: ", real_baseline_cost(x_opt, y_opt, z_opt, df_crisis));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [52]:
df_crisis_opt

Row,Time,Hydraulic,Bioenergy,Wind,Solar,Nuclear,Gas,Coal,Fuel,England_import,Spain_import,Italy_import,Switzerland_import,Germany_Belgium_import,England_export,Spain_export,Italy_export,Switzerland_export,Germany_Belgium_export
,Time,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,00:00:00,2.54996e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,223.74,61902.7,71818.6,0.0
2,00:30:00,2.54031e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,223.74,61902.7,71818.6,0.0
3,01:00:00,2.57128e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,59146.6,80055.4,0.0
4,01:30:00,2.57089e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,59146.6,80055.4,0.0
5,02:00:00,242305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,46886.4,78027.8,0.0
6,02:30:00,241608.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,46886.4,78027.8,0.0
7,03:00:00,2.33608e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,44352.0,74448.0,0.0
8,03:30:00,2.32115e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,44352.0,74448.0,0.0
9,04:00:00,218780.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53982.7,0.0,30697.9,76095.4,0.0


$\textbf{Feedback}$

No need of the $\lambda$ additional cost: even when $\alpha = 1$, there are feasible solutions. 

Next step: boost the export demand (of how much? $10\%$ ? ... $100\%$ ?)

### Sustainable Energies

In [22]:
eco_costs = compute_eco_costs(df)
carbon_footprints = compute_carbon_footprints(df)
neighbors_footprints = compute_neighbors_footprints(df);

In [23]:
neighbors_footprints

Row,Country,Carbon_footprint
,String,Float64
1,Switzerland,11.0129
2,Spain,142.357
3,England,153.825
4,Italy,165.708
5,Germany_Belgium,214.932


In [24]:
carbon_footprints[carbon_footprints[!, :Source] .== "Nuclear", :Mean_footprint][1]

6.0

In [25]:
function sustainable_df(df)
    # add carbon_footprint column for each source
    df[!, :Hydraulic_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Hydraulic", :Mean_footprint][1]
    df[!, :Nuclear_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Nuclear", :Mean_footprint][1]
    df[!, :Wind_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Wind", :Mean_footprint][1]
    df[!, :Bioenergy_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Bioenergy", :Mean_footprint][1]
    df[!, :Solar_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Solar", :Mean_footprint][1]
    df[!, :Gas_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Gas", :Mean_footprint][1]
    df[!, :Fuel_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Fuel", :Mean_footprint][1]
    df[!, :Coal_footprint] .= carbon_footprints[carbon_footprints[!, :Source] .== "Coal", :Mean_footprint][1]

    # add neighbors_footprint column for each neighbor
    df[!, :Switzerland_footprint] .= neighbors_footprints[neighbors_footprints[!, :Country] .== "Switzerland", :Carbon_footprint][1]
    df[!, :Spain_footprint] .= neighbors_footprints[neighbors_footprints[!, :Country] .== "Spain", :Carbon_footprint][1]
    df[!, :England_footprint] .= neighbors_footprints[neighbors_footprints[!, :Country] .== "England", :Carbon_footprint][1]
    df[!, :Italy_footprint] .= neighbors_footprints[neighbors_footprints[!, :Country] .== "Italy", :Carbon_footprint][1]
    df[!, :Germany_Belgium_footprint] .= neighbors_footprints[neighbors_footprints[!, :Country] .== "Germany_Belgium", :Carbon_footprint][1]

    # return the dataframe
    return df
end

sustainable_df (generic function with 1 method)

In [26]:
df_sustainable = sustainable_df(df_baseline);

In [27]:
# sustainable cost function
function lambda_sustainable_cost(x, y, z, df_base, beta)
    # for each row of the dataframe, compute a cost
    # x is the amount of energy produced by each source
    # y is the amount of energy imported from each country
    # z is the amount of energy exported to each country
    T = size(df_base, 1)

    # add production cost
    production_cost = 0
    for i in 1:size(sources, 1)
        # add cost of producing energy with each source
        production_cost += sum(x[:, i] .* ((1-beta) .* df_base[!, Symbol(sources[i] * "_cost")] .+ beta .* df_base[!, Symbol(sources[i] * "_footprint")]))
        production_cost += sum((x[:, i] - df_base[!, Symbol(sources[i])] ) .* df_base[!, Symbol("Lambda_" * sources[i])])

    end

    # add import cost
    import_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of importing energy from each neighbor
        import_cost += sum(y[:, i] .* ((1-beta) .* df_base[!, Symbol(neighbors[i] * "_Price")] .+ beta .* df_base[!, Symbol(neighbors[i] * "_footprint")]))
    end

    # add export cost
    export_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of exporting energy to each neighbor
        export_cost += sum(z[:, i] .* df_base[!, "Export_Price"])
    end    

    # return the total cost
    return production_cost + import_cost + export_cost
end


function real_sustainable_cost(x, y, z, df_base, beta)
    # for each row of the dataframe, compute a cost
    # x is the amount of energy produced by each source
    # y is the amount of energy imported from each country
    # z is the amount of energy exported to each country
    T = size(df_base, 1)

    # add production cost
    production_cost = 0
    for i in 1:size(sources, 1)
        # add cost of producing energy with each source
        production_cost += sum(x[:, i] .* ((1-beta) .* df_base[!, Symbol(sources[i] * "_cost")] .+ beta .* df_base[!, Symbol(sources[i] * "_footprint")]))
    end

    # add import cost
    import_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of importing energy from each neighbor
        import_cost += sum(y[:, i] .* ((1-beta) .* df_base[!, Symbol(neighbors[i] * "_Price")] .+ beta .* df_base[!, Symbol(neighbors[i] * "_footprint")]))
    end

    # add export cost
    export_cost = 0
    for i in 1:size(neighbors, 1)
        # add cost of exporting energy to each neighbor
        export_cost += sum(z[:, i] .* df_base[!, "Export_Price"])
    end    

    # return the total cost
    return production_cost + import_cost + export_cost
end

real_sustainable_cost (generic function with 1 method)

In [28]:
# function to build a Gurobi Optimizer to optimize the sustainable cost
function sustainable_optimizer(df_base, beta)

    # create a Gurobi Optimizer
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    # add variables
    T = size(df_base, 1)
    @variable(model, x[1:T, 1:8] >= 0) # production of energy with each source
    @variable(model, y[1:T, 1:5] >= 0) # import of energy from each neighbor
    @variable(model, z[1:T, 1:5] >= 0) # export of energy to each neighbor

    
    # add objective
    #@objective(model, Min, lambda_sustainable_cost(x, y, z, df_base, beta))
    @objective(model, Min, real_sustainable_cost(x, y, z, df_base, beta))


    # add constraints
    # Demand constraint, we need to produce enough energy for France
    @constraint(model, [t in 1:T], sum(x[t, :]) + sum(y[t, :]) - sum(z[t, :]) >= df_base[t, :Consumption])

    # maximum import available constraint
    @constraint(model, [t in 1:T, i in 1:5], y[t, i] <= df_base[t, Symbol(neighbors[i] * "_import")])

    # maximum export available constraint
    @constraint(model, [t in 1:T, i in 1:5], z[t, i] <= df_base[t, Symbol(neighbors[i] * "_export")])

    # maximum production constraint
    @constraint(model, [t in 1:T, i in 1:8], x[t, i] <= df_base[t, Symbol(sources[i] * "_cap")])

    
    # optimize
    optimize!(model)

    # return the optimizer
    return model
end

sustainable_optimizer (generic function with 1 method)

In [29]:
model_3 = sustainable_optimizer(df_sustainable, 1);

# get the optimal values
x_opt = value.(model_3[:x])
y_opt = value.(model_3[:y])
z_opt = value.(model_3[:z])

# convert the optimal values to a dataframe
df_sustainable_opt = convert_to_df(x_opt, y_opt, z_opt, df_sustainable);

# compute the cost of the optimal solution
println("Lambda baseline cost: ", lambda_baseline_cost(x_opt, y_opt, z_opt, df_sustainable))
println("Real baseline cost: ", real_baseline_cost(x_opt, y_opt, z_opt, df_sustainable))


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Lambda baseline cost: -2.2625959238743433e11
Real baseline cost: 5.193953671175134e10


In [30]:
df_sustainable_opt

Row,Time,Hydraulic,Bioenergy,Wind,Solar,Nuclear,Gas,Coal,Fuel,England_import,Spain_import,Italy_import,Switzerland_import,Germany_Belgium_import,England_export,Spain_export,Italy_export,Switzerland_export,Germany_Belgium_export
,Time,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,00:00:00,21172.8,0.0,0.0,0.0,45895.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00:30:00,21172.8,0.0,0.0,0.0,44930.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01:00:00,21172.8,0.0,0.0,0.0,42770.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01:30:00,21172.8,0.0,0.0,0.0,42731.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,02:00:00,21172.8,0.0,0.0,0.0,42235.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,02:30:00,21172.8,0.0,0.0,0.0,41538.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,03:00:00,21172.8,0.0,0.0,0.0,39652.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,03:30:00,21172.8,0.0,0.0,0.0,38159.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,04:00:00,21172.8,0.0,0.0,0.0,36831.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
